# Multi Agent Framework

In [2]:
import os
import sys
sys.path.append('../src')

from openai import OpenAI
from dotenv import load_dotenv
from agents import BaseAgent, Agent, MultiAgent

### LLM Preparation 

In [3]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

### Init Several Agents

In [4]:
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
           handoff_description = "Call this agent if a user is asking about a topic that is not handled by the activated agent.")

science_agent = Agent(base_agent = BaseAgent(client),
           name = "science agent",
           instructions = "Agent used for science questions named Albert.",
           handoff_description = "Call this agent if a user is asking about a science topic.")

music_agent = Agent(base_agent = BaseAgent(client),
           name = "music agent",
           instructions = "Agent used for music questions named Mozart.",
           handoff_description = "Call this agent if a user is asking about a music topic.")

daily_agent = Agent(base_agent = BaseAgent(client),
           name = "daily agent",
           instructions = "Agent used for daily questions named Ada.",
           handoff_description = "Call this agent if a user is asking about a daily topic.")

###  Build a Multi-Agent environment & add handoff relationships 

In [5]:
# Build a Multi-Agent environment & add handoff relationships
ma = MultiAgent(start_agent=general_agent)
ma.add_handoff_relations(from_agent=general_agent,to_agents=[science_agent,music_agent,daily_agent])
ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent])
# ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent,music_agent,daily_agent])
# ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent,science_agent,daily_agent])
# ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent,science_agent,music_agent])



### Multi-Agent handoff example

use handoff to get the agent who is suitable for the task

Then call the agent's pre-designed capabilities, such as streaming, tools, memory, reasoning, etc.

In [6]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue"}],agent=general_agent)

{'science_agent': <agents.Agent at 0x1124e6910>}

In [7]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

{'science_agent': <agents.Agent at 0x1124e6910>,
 'music_agent': <agents.Agent at 0x1124e6b50>}

### Multi-Agent dialogue example

In [8]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue"}])

[{'role': 'handoff',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <agents.Agent at 0x1124e6910>,
  'message': 'why the sky is blue'},
 {'role': 'assistant',
  'content': "The sky appears blue due to a phenomenon called Rayleigh scattering. Here's how it works:\n\n1. **Sunlight Composition**: Sunlight, or white light, is made up of various colors, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\n2. **Atmospheric Scattering**: When sunlight enters the Earth's atmosphere, it interacts with gas molecules and small particles. The shorter wavelengths of light (blue and violet) are scattered in all directions much more than the longer wavelengths (red and yellow) due to their smaller size.\n\n3. **Human Perception**: Although violet light is scattered even more than blue, our eyes are more sensitive to blue light and some violet is absorbed by the ozone layer. Thus, we perceive th

In [9]:
ma.chat(messages=[{"role": "user", "content": "who are you"}])

[{'role': 'assistant',
  'content': 'I am Albert, an AI designed to assist with science-related questions and inquiries. How can I help you today?',
  'agent_name': 'science_agent'}]

In [10]:
ma.chat(messages=[{"role": "user", "content": "recommend me some music"}])

[{'role': 'handoff',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <agents.Agent at 0x1124e4a50>,
  'message': 'recommend me some music'},
 {'role': 'handoff',
  'handoff': 'general_agent -> music_agent',
  'agent_name': 'music_agent',
  'agent': <agents.Agent at 0x1124e6b50>,
  'message': 'recommend me some music'},
 {'role': 'assistant',
  'content': 'Sure! Here are a few recommendations across different genres:\n\n### Classical\n- **Ludwig van Beethoven** - *Symphony No. 9* ("Choral")\n- **Johann Sebastian Bach** - *Goldberg Variations*\n- **Claude Debussy** - *Clair de Lune*\n\n### Jazz\n- **Miles Davis** - *Kind of Blue*\n- **John Coltrane** - *A Love Supreme*\n- **Ella Fitzgerald & Louis Armstrong** - *Ella and Louis*\n\n### Rock\n- **Led Zeppelin** - *Stairway to Heaven*\n- **The Beatles** - *Abbey Road* (album)\n- **Pink Floyd** - *Wish You Were Here*\n\n### Pop\n- **Dua Lipa** - *Future Nostalgia* (album)\n- **Taylor Swift** - *Folk

In [11]:
ma.chat(messages=[{"role": "user", "content": "dinner tonight"}])

[{'role': 'handoff',
  'handoff': 'music_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <agents.Agent at 0x1124e4a50>,
  'message': 'dinner tonight'},
 {'role': 'handoff',
  'handoff': 'general_agent -> daily_agent',
  'agent_name': 'daily_agent',
  'agent': <agents.Agent at 0x1124e6c90>,
  'message': 'dinner tonight'},
 {'role': 'assistant',
  'content': 'What are you in the mood for? I can suggest some ideas based on your preferences—like cuisine type or specific ingredients you have on hand!',
  'agent_name': 'daily_agent'}]

In [12]:
ma.chat(messages=[{"role": "user", "content": "who are you"}],agent=science_agent)

[{'role': 'assistant',
  'content': 'I am Albert, an AI designed to help answer science-related questions. How can I assist you today?',
  'agent_name': 'science_agent'}]

In [13]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

[{'role': 'handoff',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <agents.Agent at 0x1124e6910>,
  'message': 'Why is the sky blue?'},
 {'role': 'handoff',
  'handoff': 'general_agent -> music_agent',
  'agent_name': 'music_agent',
  'agent': <agents.Agent at 0x1124e6b50>,
  'message': 'Can you recommend me some music?'},
 {'role': 'assistant',
  'content': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of various colors, each corresponding to different wavelengths of light. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nAs sunlight passes through the atmosphere, the shorter wavelengths (blue and violet) are scattered in all directions by the gases and particles in the air. Although violet light is scattered even more than blue, our eyes are more sensitive to blue light and some of the violet is absorbed by the oz

### Multi-Agent tool use example

In [14]:
def get_weather(city:str)->str:
    """ 
    Get the weather for a specified city.
    """
    return f"The weather in {city} is sunny."


def get_news()->str:
    """ 
    Get the latest news.
    """
    return "The latest news is that the sun is shining."

In [15]:
daily_agent.add_tools([get_weather,get_news])

In [16]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou?"}])

[{'role': 'handoff',
  'handoff': 'general_agent -> daily_agent',
  'agent_name': 'daily_agent',
  'agent': <agents.Agent at 0x1124e6c90>,
  'message': 'what is the weather in Hangzhou?'},
 {'role': 'tool', 'content': 'The weather in Hangzhou is sunny.'}]

In [17]:
ma.chat(messages=[{"role": "user", "content": "what happened today?"}])

[{'role': 'tool', 'content': 'The latest news is that the sun is shining.'}]

In [18]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou and what happened today?"}])

[{'role': 'tool', 'content': 'The weather in Hangzhou is sunny.'},
 {'role': 'tool', 'content': 'The latest news is that the sun is shining.'}]